# 音声へのノイズ付与
音声にノイズを付与します．

## 環境構築

共有した`CS_seminar`のフォルダをダウンロードして，自分のGoogleドライブに解凍してください．

In [1]:
# Googleドライブ上のファイルを参照・編集できるように，マウントします．
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# ディレクトリを移動します(CS_seminarの中に入る)．
%cd /content/drive/MyDrive/CS_seminar

/content/drive/MyDrive/CS_seminar


In [3]:
# 環境の構築をします．詳細はniwa先輩の.ipynbを参照してください．
!apt-get install sox
!pip install librosa ipython

# # https://engineering.linecorp.com/ja/blog/voice-waveform-arbitrary-signal-to-noise-ratio-python のコードを使って、
# # SNRを指定してノイズを重畳します。
# # 量子化ビット数が16kHzじゃないとうまく動かないみたいです(今回は大丈夫!)
!git clone https://github.com/Sato-Kunihiko/audio-SNR.git

!apt-get install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base libsox3 libwavpack1
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base libsox3 libwavpack1 sox
0 upgraded, 7 newly installed, 0 to remove and 9 not upgraded.
Need to get 617 kB of archives.
After this operation, 1,764 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopencore-amrnb0 amd64 0.1.5-1 [94.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopencore-amrwb0 amd64 0.1.5-1 [49.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libsox3 amd64 14.4.2+git20190427-2+deb11u2ubuntu0.22.04.1 [240 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libso

In [15]:
import os
import re
import librosa
import IPython

## ノイズ付与
### ディレクトリ構造を確認
ノイズを付与するコマンドでファイルのパスを指定する必要があるので，音声ファイルのパスをリストで取得します．

僕のデータの場合はノイズ有無でディレクトリが異なります．なので，パスの文字列を解析して，出力ファイルの適切なパスを生成する必要があります．

話者ごとのディレクトリ構造は以下のようです．
- `__`
  - ノイズ無し，マスク無し
- `_n`
  - ノイズあり，マスク無し
- `m_`
  - ノイズ無し，マスク有り
- `mn`
  - ノイズあり，マスク有り

In [5]:
!tree ./data/wav

./data/wav
├── abe
│   ├── __
│   │   ├── iphone.wav
│   │   └── mac.wav
│   ├── m_
│   │   └── mymask.wav
│   ├── mn
│   │   └── mymask.wav
│   └── _n
│       ├── iphone.wav
│       └── mac.wav
├── ayu
│   ├── __
│   │   ├── 1_sp_b.wav
│   │   ├── 2_sp_b.wav
│   │   └── 3_sp_b.wav
│   ├── m_
│   ├── mn
│   └── _n
│       ├── 1_sp_b.wav
│       ├── 2_sp_b.wav
│       └── 3_sp_b.wav
├── hre_A
│   ├── __
│   │   ├── Ap.wav
│   │   └── As.wav
│   ├── m_
│   │   ├── mask1.wav
│   │   └── mask2.wav
│   ├── mn
│   │   ├── mask1.wav
│   │   └── mask2.wav
│   └── _n
│       ├── Ap.wav
│       └── As.wav
├── hre_B
│   ├── __
│   │   ├── Bp.wav
│   │   └── Bs.wav
│   ├── m_
│   ├── mn
│   └── _n
│       ├── Bp.wav
│       └── Bs.wav
├── hre_C
│   ├── __
│   │   ├── Cp.wav
│   │   └── Cs.wav
│   ├── m_
│   ├── mn
│   └── _n
│       ├── Cp.wav
│       └── Cs.wav
├── ktg
│   ├── __
│   │   ├── 1_sp_b.wav
│   │   ├── 2_sp_b.wav
│   │   ├── 3_sp_b.wav
│   │   ├── 4_pc_c.wav
│   │   ├── 5_pc_c.wav
│  

### ファイルパスを取得

In [6]:
!echo $PWD

/content/drive/MyDrive/CS_seminar


In [7]:
%cd ./data/wav
WAV_ROOT = %pwd # 音声データのルートパス

/content/drive/MyDrive/CS_seminar/data/wav


In [9]:
# 話者ディレクトリのリストを作成
# 各話者/各サブディレクトリまで潜り，ファイルパスを取得，全体のファイルパスのリストにappendする
vanilla_wav_path = []

speakers = os.listdir(WAV_ROOT)
for speaker in speakers:
  # WAV_ROOT + '\' + {話者ディレクトリ名} をos.listdir()に与えると，そこにあるアイテムを取得できる．
  spk_dir = WAV_ROOT + '/' + speaker
  subdirs = os.listdir(spk_dir)
  for subdir in subdirs:
    # ここで取得したいのはノイズが乗っていないファイルのパスなので，subdirがノイズ付きのディレクトリかどうか判定する．
    if subdir[1] == 'n':
      continue
    pwd = spk_dir + '/' + subdir
    wav_names = os.listdir(pwd)
    for fname in wav_names:
      wav_path = pwd + '/' +  fname
      vanilla_wav_path.append(wav_path)

# 確認
print(*vanilla_wav_path, sep='\n')

/content/drive/MyDrive/CS_seminar/data/wav/abe/__/mac.wav
/content/drive/MyDrive/CS_seminar/data/wav/abe/__/iphone.wav
/content/drive/MyDrive/CS_seminar/data/wav/abe/m_/mymask.wav
/content/drive/MyDrive/CS_seminar/data/wav/tmng/__/iPhone.wav
/content/drive/MyDrive/CS_seminar/data/wav/tmng/__/PC.wav
/content/drive/MyDrive/CS_seminar/data/wav/tmng/m_/mask.wav
/content/drive/MyDrive/CS_seminar/data/wav/hre_A/__/Ap.wav
/content/drive/MyDrive/CS_seminar/data/wav/hre_A/__/As.wav
/content/drive/MyDrive/CS_seminar/data/wav/hre_A/m_/mask2.wav
/content/drive/MyDrive/CS_seminar/data/wav/hre_A/m_/mask1.wav
/content/drive/MyDrive/CS_seminar/data/wav/shina/__/1_sp_a.wav
/content/drive/MyDrive/CS_seminar/data/wav/shina/__/3_sp_a.wav
/content/drive/MyDrive/CS_seminar/data/wav/shina/__/2_sp_a.wav
/content/drive/MyDrive/CS_seminar/data/wav/ktg/__/1_sp_b.wav
/content/drive/MyDrive/CS_seminar/data/wav/ktg/__/6_pc_c.wav
/content/drive/MyDrive/CS_seminar/data/wav/ktg/__/3_sp_b.wav
/content/drive/MyDrive/CS_

### shellコマンドでノイズを付与する

niwa先輩のコードを参考にします．
ノイズを付与したファイルの格納先は,話者内のサブディレクトリの2文字目をnにしたディレクトリ内であればよいです．

In [10]:
!echo $PWD

/content/drive/MyDrive/CS_seminar/data/wav


In [11]:
# ノイズをかける処理をまとめておく
# 自分はホワイトノイズ, SNR=5でやってみます．
def apply_white_noise(src_path, out_path):
  # ホワイトノイズ生成
  !sox {src_path} ../wnz.wav synth whitenoise vol 1
  # mix
  !python3 ../../audio-SNR/create_mixed_audio_file.py --clean_file {src_path} --noise_file ../wnz.wav --output_mixed_file {out_path} --snr 5
  # ファイル名に'('があるとbashコマンド的に不適なようです．

In [12]:
for path in vanilla_wav_path:
  # 話者単位のサブディレクトリを取得します．
  # サブディレクトリ名が"__" -> "_n", "m_" -> "mn"にそれぞれノイズ付き音声を格納します．
  path_components = path.split('/')
  # "__" or "m_"のディレクトリはpath_components[-2]
  # 2文字目を'n'にする
  path_components[-2] = path_components[-2][0] + 'n'
  # pathを再構築
  nz_path = '/'.join(path_components)

  # ノイズをかけたい音声のパスと，ノイズをかけた後の音声のパスが得られたので，コマンドでノイズをかける．
  apply_white_noise(path, nz_path)

sox WARN dither: dither clipped 1 samples; decrease volume?
sox WARN dither: dither clipped 1 samples; decrease volume?
sox WARN dither: dither clipped 1 samples; decrease volume?


## 確認
ノイズがかかっているかどうか，また，ディレクトリ構造が壊れていないか，一応確認しておきます．

In [13]:
!echo $PWD

/content/drive/MyDrive/CS_seminar/data/wav


In [14]:
!tree

.
├── abe
│   ├── __
│   │   ├── iphone.wav
│   │   └── mac.wav
│   ├── m_
│   │   └── mymask.wav
│   ├── mn
│   │   └── mymask.wav
│   └── _n
│       ├── iphone.wav
│       └── mac.wav
├── ayu
│   ├── __
│   │   ├── 1_sp_b.wav
│   │   ├── 2_sp_b.wav
│   │   └── 3_sp_b.wav
│   ├── m_
│   ├── mn
│   └── _n
│       ├── 1_sp_b.wav
│       ├── 2_sp_b.wav
│       └── 3_sp_b.wav
├── hre_A
│   ├── __
│   │   ├── Ap.wav
│   │   └── As.wav
│   ├── m_
│   │   ├── mask1.wav
│   │   └── mask2.wav
│   ├── mn
│   │   ├── mask1.wav
│   │   └── mask2.wav
│   └── _n
│       ├── Ap.wav
│       └── As.wav
├── hre_B
│   ├── __
│   │   ├── Bp.wav
│   │   └── Bs.wav
│   ├── m_
│   ├── mn
│   └── _n
│       ├── Bp.wav
│       └── Bs.wav
├── hre_C
│   ├── __
│   │   ├── Cp.wav
│   │   └── Cs.wav
│   ├── m_
│   ├── mn
│   └── _n
│       ├── Cp.wav
│       └── Cs.wav
├── ktg
│   ├── __
│   │   ├── 1_sp_b.wav
│   │   ├── 2_sp_b.wav
│   │   ├── 3_sp_b.wav
│   │   ├── 4_pc_c.wav
│   │   ├── 5_pc_c.wav
│   │   └── 

In [16]:
# 加工前の音声の確認
x, sr = librosa.load("./ymgt/__/0_sp_a.wav")
display(IPython.display.Audio(x, rate=sr))

In [17]:
# 加工後の音声の確認
x, sr = librosa.load("./ymgt/_n/0_sp_a.wav")
display(IPython.display.Audio(x, rate=sr))